# MOVIES PROJECT

necesary imports

In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

get responses from lots of urld urls <br>
starting with: <br>
https://www.imdb.com/...&start=1 <br>
up to: <br>
https://www.imdb.com/...&start=6651

In [28]:
URL = "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&countries=us&sort=user_rating,desc&start="

responses = []
for start_pt in range(1, 6652, 50):
# for start_pt in range(1, 150, 50):
    responses.append(requests.get(URL + str(start_pt)))
len(responses)

134

sopify all of them

In [29]:
soups = []
for response in responses:
    soups.append(BeautifulSoup(response.text, 'html.parser'))
len(soups)

134

## RELEASE DATES

date cleaning function

In [30]:
def extract_int(string):
    return re.search(r"\d+", string).group()

traverse the soups in search for dates, create a list of them, serialise it

In [31]:
movie_release_dates = []
for soup in soups:
    movie_release_dates += soup.findAll('span', class_= 'lister-item-year text-muted unbold')

list_release_dates = []
for str_date in movie_release_dates:
    list_release_dates.append( extract_int(str_date.text) )

rel_dates_series = pd.Series(list_release_dates, name="Release Date")
rel_dates_series

0       1994
1       1972
2       2008
3       1974
4       1957
        ... 
6670    2010
6671    2006
6672    1989
6673    1966
6674    2014
Name: Release Date, Length: 6675, dtype: object

## RUNTIMES

create a time-cleaning function

In [32]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

traverse the soups in search for runtimes, create a list of them, serialise it

In [33]:
runtimes = []
for soup in soups:
    runtimes += soup.findAll('span', class_="runtime")

movie_runtimes = []
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

runtimes_series

0       142
1       175
2       152
3       202
4        96
       ... 
6670    105
6671     91
6672     99
6673     70
6674     79
Name: Runtime, Length: 6675, dtype: int64

## MOVIE NAMES

traverse the soups in search for movie names, create a list of them, serialise it

In [34]:
movie_names = []
for soup in soups:
    movie_names += soup.find_all(name="h3", class_="lister-item-header")

names_list = []
for mn in movie_names:
    names_list.append(list(mn)[3].text)

names_series = pd.Series(names_list, name="Movie name")
names_series

0             Skazani na Shawshank
1                 Ojciec chrzestny
2                   Mroczny Rycerz
3              Ojciec chrzestny II
4         Dwunastu gniewnych ludzi
                   ...            
6670                    Ptakodemia
6671                  Pledge This!
6672               Ahoj dziewczyny
6673    Manos - Ręce przeznaczenia
6674              Saving Christmas
Name: Movie name, Length: 6675, dtype: object

## MOVIE TYPES

traverse the soups in search for movie names, clean them, create a list of them and serialise it

In [35]:
genres = []

for soup in soups:
    genres += soup.find_all(name="span", class_="genre")

genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)

genres_series = pd.Series(genres_list, name="Genre")
genres_series

0                      Drama
1               Crime, Drama
2       Action, Crime, Drama
3               Crime, Drama
4               Crime, Drama
                ...         
6670        Horror, Thriller
6671                  Comedy
6672                  Comedy
6673                  Horror
6674          Comedy, Family
Name: Genre, Length: 6675, dtype: object

## RATING

traverse the soup, get stuff from it and serialise it in 2 steps

In [36]:
# <div class="inline-block ratings-imdb-rating" name="ir" data-value="8,3">
ratings = []
for soup in soups:
    ratings += soup.find_all(name="div", attrs={'name':'ir'})

ratings_series = pd.Series([ float(rt.text.strip()) for rt in ratings ], name="Rating")
ratings_series

0       9.3
1       9.2
2       9.0
3       9.0
4       9.0
       ... 
6670    1.8
6671    1.8
6672    1.8
6673    1.8
6674    1.4
Name: Rating, Length: 6675, dtype: float64

## PROFIT!

In [37]:
all_series = [names_series, ratings_series, rel_dates_series, runtimes_series, genres_series]

df = pd.concat(all_series, axis=1)

# %timeit df.loc[df.loc[:,'Genre'].str.contains('Action')]
# %timeit df.loc[df['Genre'].str.contains('Action')]
# %timeit df.loc[ [('Action' in genre) for genre in df.loc[:,'Genre']]]
# %timeit df.loc[ [('Action' in genre) for genre in df['Genre']]]


In [38]:
df

,Movie name,Rating,Release Date,Runtime,Genre
0,Skazani na Shawshank,9.3,1994,142,Drama
1,Ojciec chrzestny,9.2,1972,175,"Crime, Drama"
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
3,Ojciec chrzestny II,9.0,1974,202,"Crime, Drama"
4,Dwunastu gniewnych ludzi,9.0,1957,96,"Crime, Drama"
...,...,...,...,...,...
6670,Ptakodemia,1.8,2010,105,"Horror, Thriller"
6671,Pledge This!,1.8,2006,91,Comedy
6672,Ahoj dziewczyny,1.8,1989,99,Comedy
6673,Manos - Ręce przeznaczenia,1.8,1966,70,Horror


extract action movies

In [39]:
action = df.loc[ [('Action' in genre) for genre in df['Genre']]]
action

,Movie name,Rating,Release Date,Runtime,Genre
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
5,Władca pierścieni: Powrót króla,8.9,2003,201,"Action, Adventure, Drama"
8,Incepcja,8.8,2010,148,"Action, Adventure, Sci-Fi"
10,Władca pierścieni: Drużyna Pierścienia,8.8,2001,178,"Action, Adventure, Drama"
12,Władca pierścieni: Dwie wieże,8.7,2002,179,"Action, Adventure, Drama"
...,...,...,...,...,...
6649,Małolaty ninja w lunaparku,2.9,1998,93,"Action, Adventure, Comedy"
6655,Dragonball: Ewolucja,2.5,2009,85,"Action, Adventure, Fantasy"
6657,Bitwa o Ziemię,2.5,2000,118,"Action, Adventure, Sci-Fi"
6659,Alone in the Dark: Wyspa cienia,2.4,2005,99,"Action, Horror, Sci-Fi"
